In [1]:
import pandas as pd
import numpy as np

In [2]:
basic = pd.read_pickle('../data/pkl/raw_games_5yrs.pkl')
advanced = pd.read_pickle('../data/pkl/boxscores_advanced_team_part1.pkl')

In [3]:
basic = basic.sort_values(by=['GAME_DATE', 'GAME_ID'], ascending=False).reset_index(drop=True)

In [4]:
basic.columns

Index(['SEASON_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID',
       'GAME_DATE', 'MATCHUP', 'WL', 'MIN', 'PTS', 'FGM', 'FGA', 'FG_PCT',
       'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB',
       'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PLUS_MINUS', 'HOME_TEAM'],
      dtype='object')

In [5]:
games_df = basic[['TEAM_ID', 'TEAM_ABBREVIATION', 'GAME_ID', 'GAME_DATE', 'HOME_TEAM', 'PLUS_MINUS']].copy()
games_df.head()

,TEAM_ID,TEAM_ABBREVIATION,GAME_ID,GAME_DATE,HOME_TEAM,PLUS_MINUS
0,1610612749,MIL,22200973,2023-03-05,0,6.0
1,1610612764,WAS,22200973,2023-03-05,1,-6.0
2,1610612746,LAC,22200970,2023-03-05,1,6.0
3,1610612763,MEM,22200970,2023-03-05,0,-6.0
4,1610612738,BOS,22200969,2023-03-05,1,-2.0


In [6]:
advanced.head()

,GAME_ID,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CITY,MIN,E_OFF_RATING,OFF_RATING,E_DEF_RATING,DEF_RATING,...,TM_TOV_PCT,EFG_PCT,TS_PCT,USG_PCT,E_USG_PCT,E_PACE,PACE,PACE_PER40,POSS,PIE
0,0022200963,1610612754,Pacers,IND,Indiana,240:00,128.8,133.0,127.1,131.2,...,9.6,0.596,0.619,1.0,0.196,96.52,93.5,77.92,94,0.502
1,0022200963,1610612741,Bulls,CHI,Chicago,240:00,127.1,131.2,128.8,133.0,...,16.1,0.676,0.718,1.0,0.198,96.52,93.5,77.92,93,0.498
0,0022200962,1610612756,Suns,PHX,Phoenix,240:00,130.0,134.0,127.6,128.6,...,9.3,0.618,0.650,1.0,0.200,99.36,97.5,81.25,97,0.539
1,0022200962,1610612742,Mavericks,DAL,Dallas,240:00,127.6,128.6,130.0,134.0,...,10.2,0.591,0.672,1.0,0.197,99.36,97.5,81.25,98,0.461
0,0022200964,1610612747,Lakers,LAL,Los Angeles,240:00,109.0,114.1,100.1,106.1,...,9.1,0.500,0.540,1.0,0.197,104.30,99.0,82.50,99,0.556


In [7]:
advanced.columns

Index(['GAME_ID', 'TEAM_ID', 'TEAM_NAME', 'TEAM_ABBREVIATION', 'TEAM_CITY',
       'MIN', 'E_OFF_RATING', 'OFF_RATING', 'E_DEF_RATING', 'DEF_RATING',
       'E_NET_RATING', 'NET_RATING', 'AST_PCT', 'AST_TOV', 'AST_RATIO',
       'OREB_PCT', 'DREB_PCT', 'REB_PCT', 'E_TM_TOV_PCT', 'TM_TOV_PCT',
       'EFG_PCT', 'TS_PCT', 'USG_PCT', 'E_USG_PCT', 'E_PACE', 'PACE',
       'PACE_PER40', 'POSS', 'PIE'],
      dtype='object')

In [8]:
columns_to_drop = ['TEAM_CITY', 'MIN', 'E_OFF_RATING', 'E_DEF_RATING', 
                   'E_NET_RATING', 'AST_RATIO', 'E_TM_TOV_PCT', 'USG_PCT', 
                   'E_USG_PCT', 'E_PACE', 'PACE_PER40', 'PIE']

In [9]:
advanced = advanced.drop(columns=columns_to_drop)
advanced

,GAME_ID,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,OFF_RATING,DEF_RATING,NET_RATING,AST_PCT,AST_TOV,OREB_PCT,DREB_PCT,REB_PCT,TM_TOV_PCT,EFG_PCT,TS_PCT,PACE,POSS
0,0022200963,1610612754,Pacers,IND,133.0,131.2,1.8,0.563,3.00,0.319,0.742,0.487,9.6,0.596,0.619,93.5,94
1,0022200963,1610612741,Bulls,CHI,131.2,133.0,-1.8,0.400,1.20,0.258,0.681,0.513,16.1,0.676,0.718,93.5,93
0,0022200962,1610612756,Suns,PHX,134.0,128.6,5.4,0.551,3.00,0.317,0.829,0.573,9.3,0.618,0.650,97.5,97
1,0022200962,1610612742,Mavericks,DAL,128.6,134.0,-5.4,0.395,1.50,0.171,0.683,0.427,10.2,0.591,0.672,97.5,98
0,0022200964,1610612747,Lakers,LAL,114.1,106.1,8.1,0.690,3.22,0.309,0.754,0.536,9.1,0.500,0.540,99.0,99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,0022100003,1610612766,Hornets,CHA,116.0,114.0,2.0,0.630,3.63,0.333,0.745,0.513,7.5,0.491,0.517,106.5,106
0,0022100002,1610612747,Lakers,LAL,101.8,107.1,-5.3,0.467,1.17,0.189,0.759,0.477,16.1,0.553,0.551,112.5,112
1,0022100002,1610612744,Warriors,GSW,107.1,101.8,5.3,0.732,1.76,0.241,0.811,0.523,15.0,0.516,0.570,112.5,113
0,0022100001,1610612749,Bucks,MIL,124.5,102.0,22.5,0.521,3.13,0.293,0.824,0.541,7.8,0.538,0.562,102.0,102


In [10]:
advanced['GAME_ID'] = advanced['GAME_ID'].astype('int32')

In [11]:
advanced = advanced.merge(games_df.drop(columns=['TEAM_ID']), on=['GAME_ID', 'TEAM_ABBREVIATION'])

In [12]:
advanced.head()

,GAME_ID,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,OFF_RATING,DEF_RATING,NET_RATING,AST_PCT,AST_TOV,OREB_PCT,DREB_PCT,REB_PCT,TM_TOV_PCT,EFG_PCT,TS_PCT,PACE,POSS,GAME_DATE,HOME_TEAM,PLUS_MINUS
0,22200963,1610612754,Pacers,IND,133.0,131.2,1.8,0.563,3.00,0.319,0.742,0.487,9.6,0.596,0.619,93.5,94,2023-03-05,0,6.4
1,22200963,1610612741,Bulls,CHI,131.2,133.0,-1.8,0.400,1.20,0.258,0.681,0.513,16.1,0.676,0.718,93.5,93,2023-03-05,1,-3.0
2,22200962,1610612756,Suns,PHX,134.0,128.6,5.4,0.551,3.00,0.317,0.829,0.573,9.3,0.618,0.650,97.5,97,2023-03-05,0,4.0
3,22200962,1610612742,Mavericks,DAL,128.6,134.0,-5.4,0.395,1.50,0.171,0.683,0.427,10.2,0.591,0.672,97.5,98,2023-03-05,1,-4.0
4,22200964,1610612747,Lakers,LAL,114.1,106.1,8.1,0.690,3.22,0.309,0.754,0.536,9.1,0.500,0.540,99.0,99,2023-03-05,1,8.0


Reorder the data frame

#### DROP ROWS THAT DONT HAVE A MATCHING GAME ID

In [13]:
value_counts = advanced['GAME_ID'].value_counts()

In [14]:
unique_values = value_counts[value_counts == 1].index.tolist()

In [15]:
advanced = advanced[~advanced['GAME_ID'].isin(unique_values)].reset_index(drop=True)

In [16]:
advanced_desc = advanced.sort_values(by=['GAME_DATE'], ascending=True).copy()

In [110]:
rolling_all_test = advanced.sort_values(by=['GAME_DATE', 'GAME_ID'], ascending=True).copy()

#### Data Frame is ready to be preprocessed

In [18]:
non_eng_features = ['TEAM_ABBREVIATION', 'GAME_ID', 'TEAM_ID', 'TEAM_NAME', 
                        'GAME_DATE', 'HOME_TEAM', 'PLUS_MINUS']

In [19]:
eng_features = advanced_desc.drop(columns=non_eng_features).columns.tolist()

In [20]:
eng_features

['OFF_RATING',
 'DEF_RATING',
 'NET_RATING',
 'AST_PCT',
 'AST_TOV',
 'OREB_PCT',
 'DREB_PCT',
 'REB_PCT',
 'TM_TOV_PCT',
 'EFG_PCT',
 'TS_PCT',
 'PACE',
 'POSS']

In [78]:
advanced_desc[eng_features + ["TEAM_ABBREVIATION", 'GAME_DATE', 'GAME_ID']].sort_values(["TEAM_ABBREVIATION", 'GAME_DATE'], ascending=[True, False]).head(10)

,OFF_RATING,DEF_RATING,NET_RATING,AST_PCT,AST_TOV,OREB_PCT,DREB_PCT,REB_PCT,TM_TOV_PCT,EFG_PCT,TS_PCT,PACE,POSS,TEAM_ABBREVIATION,GAME_DATE,GAME_ID
23,110.1,117.0,-6.9,0.595,1.38,0.250,0.674,0.460,16.2,0.525,0.591,99.5,99,ATL,2023-03-04,22200958
39,125.2,107.8,17.5,0.577,1.76,0.366,0.723,0.557,16.5,0.637,0.658,103.0,103,ATL,2023-03-03,22200947
83,119.6,125.3,-5.7,0.488,2.33,0.333,0.600,0.465,9.3,0.532,0.572,96.0,97,ATL,2023-02-28,22200923
115,125.2,121.0,4.3,0.444,1.25,0.333,0.795,0.554,15.5,0.589,0.630,104.0,103,ATL,2023-02-26,22200911
141,132.0,115.5,16.5,0.560,2.00,0.222,0.792,0.548,13.6,0.676,0.715,103.0,103,ATL,2023-02-24,22200896
187,100.0,119.6,-19.6,0.625,5.00,0.169,0.712,0.423,5.0,0.438,0.475,101.5,101,ATL,2023-02-15,22200876
209,127.8,132.1,-4.3,0.491,2.17,0.364,0.818,0.534,11.1,0.542,0.589,108.5,108,ATL,2023-02-13,22200857
243,117.9,100.0,17.9,0.580,2.90,0.321,0.673,0.495,9.4,0.534,0.561,106.0,106,ATL,2023-02-11,22200850
281,114.9,104.9,9.9,0.535,1.21,0.473,0.800,0.629,18.8,0.505,0.543,101.5,101,ATL,2023-02-09,22200832
305,111.5,122.1,-10.6,0.789,2.50,0.300,0.667,0.480,12.5,0.518,0.557,95.5,96,ATL,2023-02-07,22200818


In [113]:
def roll(df1, df2, roll_number = 10, procedure = '', suff = '_Roll', selected_columns=[]):
    
    df_rolling = df1[selected_columns + ["TEAM_ABBREVIATION"]]
    df_rolling = df_rolling.groupby(["TEAM_ABBREVIATION"], group_keys=False)
    df_all_teams = df2[selected_columns]
    
    def find_team_averages(team):
        return team.rolling(roll_number, closed='left').mean()

    def find_team_medians(team):
        return team.rolling(roll_number, closed='left').median()
    
    def find_team_stds(team):
        return team.rolling(roll_number, closed='left').std()

    if procedure == 'median':
        df_rolling = df_rolling.apply(find_team_medians)
        df_all_teams = df_all_teams.apply(find_team_medians)
    elif procedure == 'std':
        df_rolling = df_rolling.apply(find_team_stds)
        df_all_teams = df_all_teams.apply(find_team_stds)
    else:
        procedure = 'mean'
        df_rolling = df_rolling.apply(find_team_averages)
        df_all_teams = df_all_teams.apply(find_team_averages)
           
    df_rolling = df_rolling[selected_columns]
    df_rolling = df_rolling.sort_index()
    df_all_teams = df_all_teams[selected_columns]
    df_all_teams= df_all_teams.sort_index()
    
    new_column_names = {}
    for col in df_rolling.columns:
        new_column_names[col] = col + suff + '_' + procedure
        
    new_all = {}
    for col in df_all_teams.columns:
        new_all[col] = col + '_all_' + procedure

    df_rolling = df_rolling.rename(columns=new_column_names)
    df_all_teams = df_all_teams.rename(columns=new_all)
    return df_rolling, df_all_teams

In [114]:
df_roll, df_roll_all = roll(df1 = advanced_desc, df2 = rolling_all_test, roll_number=5, procedure='mean', selected_columns=eng_features)
# df_test2 = roll(df = advanced_desc, roll_number=5, procedure='median', selected_columns=eng_features)
# df_test3 = roll(df = advanced_desc, roll_number=5, procedure='std', selected_columns=eng_features)

/tmp/ipykernel_3166/4177443528.py:8: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['TEAM_ABBREVIATION'], dtype='object')
  return team.rolling(roll_number, closed='left').mean()


In [115]:
df_roll_all.sort_index().head(10)

,OFF_RATING_all_mean,DEF_RATING_all_mean,NET_RATING_all_mean,AST_PCT_all_mean,AST_TOV_all_mean,OREB_PCT_all_mean,DREB_PCT_all_mean,REB_PCT_all_mean,TM_TOV_PCT_all_mean,EFG_PCT_all_mean,TS_PCT_all_mean,PACE_all_mean,POSS_all_mean
0,120.08,122.02,-1.94,0.5810,1.988,0.2876,0.7124,0.4988,14.24,0.5714,0.6120,99.468,107.4
1,119.18,119.92,-0.74,0.5328,2.078,0.2902,0.7472,0.5112,12.94,0.5538,0.5908,99.068,106.8
2,114.72,120.66,-5.94,0.5578,1.600,0.3026,0.6876,0.4850,14.62,0.5448,0.5814,99.968,107.8
3,119.58,118.90,0.66,0.5620,1.588,0.3394,0.7330,0.5380,15.00,0.5604,0.6082,99.668,107.8
4,120.72,120.14,0.58,0.6056,1.884,0.2990,0.7284,0.5148,14.00,0.5826,0.6246,103.000,103.0
5,119.24,119.58,-0.34,0.5928,1.904,0.3058,0.7062,0.5016,13.44,0.5676,0.6066,101.634,105.6
6,112.94,116.12,-3.16,0.5980,1.976,0.2234,0.7630,0.4938,12.70,0.5522,0.5912,102.400,102.4
7,120.96,119.02,1.94,0.6040,2.036,0.2672,0.7328,0.5012,13.06,0.5854,0.6272,102.800,102.8
8,105.70,107.30,-1.62,0.6460,2.128,0.1998,0.7876,0.4928,11.68,0.5006,0.5450,101.000,101.0
9,111.78,105.84,5.94,0.6076,2.196,0.2094,0.8004,0.5150,11.36,0.5392,0.5840,101.900,102.0


In [138]:
rolling_all_test['OFF_RATING'].iloc[1:5]

4715    102.0
4712    101.8
4713    107.1
4710    116.0
Name: OFF_RATING, dtype: float64

In [49]:
advanced = advanced.merge(df_test, left_index=True, right_index=True)
advanced = advanced.merge(df_test2, left_index=True, right_index=True)
advanced = advanced.merge(df_test3, left_index=True, right_index=True)

In [50]:
#advanced_desc = advanced_desc.groupby('TEAM_ABBREVIATION', as_index=False, group_keys=False)[eng_features].rolling(5).mean()
#advanced_desc.drop(columns=['TEAM_ABBREVIATION']).sort_index()
#advanced[eng_features] = advanced_desc.drop(columns=['TEAM_ABBREVIATION']).sort_index()

In [51]:
advanced = advanced.sort_values(by=['GAME_DATE', 'GAME_ID', 'HOME_TEAM'], ascending=False)

In [82]:
advanced[advanced['GAME_ID'] == 22200958]['OFF_RATING_Roll_mean_x']

26    106.82
27    120.40
Name: OFF_RATING_Roll_mean_x, dtype: float64

In [85]:
adv_home = advanced.iloc[::2].copy()
adv_away = advanced.iloc[1::2].copy()

In [86]:
columns_away = {}
columns_home = {}
columns_to_merge = []
for column in advanced.columns:
    if column == 'GAME_ID' or column == 'PLUS_MINUS' or column == 'GAME_DATE':
        continue
    columns_to_merge.append(column + '_a')
    columns_away[column] = column + '_a'
    columns_home[column] = column + '_h'

In [87]:
columns_away

{'TEAM_ID': 'TEAM_ID_a',
 'TEAM_NAME': 'TEAM_NAME_a',
 'TEAM_ABBREVIATION': 'TEAM_ABBREVIATION_a',
 'OFF_RATING': 'OFF_RATING_a',
 'DEF_RATING': 'DEF_RATING_a',
 'NET_RATING': 'NET_RATING_a',
 'AST_PCT': 'AST_PCT_a',
 'AST_TOV': 'AST_TOV_a',
 'OREB_PCT': 'OREB_PCT_a',
 'DREB_PCT': 'DREB_PCT_a',
 'REB_PCT': 'REB_PCT_a',
 'TM_TOV_PCT': 'TM_TOV_PCT_a',
 'EFG_PCT': 'EFG_PCT_a',
 'TS_PCT': 'TS_PCT_a',
 'PACE': 'PACE_a',
 'POSS': 'POSS_a',
 'HOME_TEAM': 'HOME_TEAM_a',
 'OFF_RATING_Roll_mean': 'OFF_RATING_Roll_mean_a',
 'DEF_RATING_Roll_mean': 'DEF_RATING_Roll_mean_a',
 'NET_RATING_Roll_mean': 'NET_RATING_Roll_mean_a',
 'AST_PCT_Roll_mean': 'AST_PCT_Roll_mean_a',
 'AST_TOV_Roll_mean': 'AST_TOV_Roll_mean_a',
 'OREB_PCT_Roll_mean': 'OREB_PCT_Roll_mean_a',
 'DREB_PCT_Roll_mean': 'DREB_PCT_Roll_mean_a',
 'REB_PCT_Roll_mean': 'REB_PCT_Roll_mean_a',
 'TM_TOV_PCT_Roll_mean': 'TM_TOV_PCT_Roll_mean_a',
 'EFG_PCT_Roll_mean': 'EFG_PCT_Roll_mean_a',
 'TS_PCT_Roll_mean': 'TS_PCT_Roll_mean_a',
 'PACE_Roll_

In [88]:
adv_away.rename(columns=columns_away, inplace=True)
adv_home.rename(columns=columns_home, inplace=True)

In [89]:
columns_to_merge.append('GAME_ID')

In [90]:
merged_df = adv_home.merge(adv_away[columns_to_merge], on=['GAME_ID'])

In [91]:
merged_df = merged_df.dropna()

#### Now the data is ready for scaling and one hot encoding

In [92]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

In [93]:
X_features_num = [col for col in merged_df.columns if 'GAME_ID' not in col 
                     and 'GAME_DATE' not in col 
                     and 'TEAM_ID' not in col 
                     and 'TEAM_NAME' not in col
                     and 'TEAM_ABBREVIATION' not in col
                     and 'PLUS_MINUS' not in col
                     and 'HOME_TEAM' not in col]

In [94]:
X_features_cat = ['TEAM_ABBREVIATION_h', 'TEAM_ABBREVIATION_a']

In [95]:
preproc_data = merged_df.copy()

In [96]:
scaler = MinMaxScaler()
preproc_data[X_features_num] = scaler.fit_transform(preproc_data[X_features_num])

In [97]:
ohe = OneHotEncoder(sparse=False)
ohe.fit(preproc_data[X_features_cat])
cols = [str(team) +'_h' for team in ohe.categories_[0]] + [str(team) +'_a' for team in ohe.categories_[1]]

/home/alec/.pyenv/versions/nba_betting_analysis/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [98]:
preproc_data[cols]=ohe.transform(preproc_data[X_features_cat])

In [99]:
X_features = [col for col in preproc_data.columns if 'GAME_ID' not in col 
                     and 'GAME_DATE' not in col 
                     and 'TEAM_ID' not in col 
                     and 'TEAM_NAME' not in col
                     and 'TEAM_ABBREVIATION' not in col
                     and 'PLUS_MINUS' not in col
                     and 'HOME_TEAM' not in col]

In [100]:
X_preproc = preproc_data[X_features]
y = preproc_data['PLUS_MINUS']

In [104]:
len(preproc_data['TEAM_NAME_h'].unique())

30

In [ ]:
#preproc_data.to_pickle('advanced_preproc_part1.pkl')

In [46]:
#X_preproc.to_pickle('X_preproc_rolling_advanced_part2.pkl')

In [47]:
#y.to_pickle('y_preproc_rolling_advanced_part2.pkl')